<a href="https://colab.research.google.com/github/siren7075/OpenAI_Embeddings_Public/blob/main/OpenAI_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Open AI

In [ ]:
!pip install --upgrade openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# imports
from typing import List

import pandas as pd
import pickle

from openai.embeddings_utils import (
    get_embedding,
    distances_from_embeddings,
    tsne_components_from_embeddings,
    chart_from_components,
    indices_of_nearest_neighbors_from_distances,
)

In [ ]:
openai.api_key = ""

In [ ]:
df = pd.read_csv("/content/Cusips_Combined.csv")
df.head(5)

,cusip,security_name,coupon,maturity_date,issuer,instrument_type_gps_name,industry_group_lehman2_name,sector_lehman2_name,currency_code,country,rating_moody_or_sp,liquidity,bids,asks,combined
0,000000299,S&P 400 MIDCAP INDEX,NaN,NaN,SPXT,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR,NaN,NaN,NaN,"S&P 400 MIDCAP INDEX,NULL,NULL,SPXT,EMI,UNKNOW..."
1,000000711,S&P 500 INDEX,0.0,NaN,SPXT,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR,NaN,NaN,NaN,"S&P 500 INDEX,0,NULL,SPXT,EMI,UNKNOWN,UNKNOWN,..."
2,000000737,DOW JONES INDUS. AVG,0.0,NaN,INDU,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR,NaN,NaN,NaN,"DOW JONES INDUS. AVG,0,NULL,INDU,EMI,UNKNOWN,U..."
3,000001KP1,RUSSELL 1000 INDEX,0.0,NaN,RLG,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR,NaN,NaN,NaN,"RUSSELL 1000 INDEX,0,NULL,RLG,EMI,UNKNOWN,UNKN..."
4,000001KR7,RUSSEL 3000 INDEX,0.0,NaN,RLG,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR,NaN,NaN,NaN,"RUSSEL 3000 INDEX,0,NULL,RLG,EMI,UNKNOWN,UNKNO..."


In [ ]:
len(df)

10102

In [ ]:
df_1000 = df.head(1000)
len(df_1000)

1000

In [ ]:
df['davinci_similarity'] = df_1000.combined.apply(lambda x: get_embedding(x, engine='text-similarity-davinci-001'))
df.to_csv('/Cusips_Embeddings.csv')

In [ ]:
df_embeddings = pd.read_csv("/Cusips_Embeddings.csv")
df_embeddings.head(5)

,Unnamed: 0,cusip,security_name,coupon,maturity_date,issuer,instrument_type_gps_name,industry_group_lehman2_name,sector_lehman2_name,currency_code,country,rating_moody_or_sp,liquidity,bids,asks,combined,davinci_similarity
0,0,000000299,S&P 400 MIDCAP INDEX,NaN,NaN,SPXT,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR,NaN,NaN,NaN,"S&P 400 MIDCAP INDEX,NULL,NULL,SPXT,EMI,UNKNOW...","[-0.009326230734586716, 0.007269254419952631, ..."
1,1,000000711,S&P 500 INDEX,0.0,NaN,SPXT,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR,NaN,NaN,NaN,"S&P 500 INDEX,0,NULL,SPXT,EMI,UNKNOWN,UNKNOWN,...","[-0.010450955480337143, 0.006831851787865162, ..."
2,2,000000737,DOW JONES INDUS. AVG,0.0,NaN,INDU,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR,NaN,NaN,NaN,"DOW JONES INDUS. AVG,0,NULL,INDU,EMI,UNKNOWN,U...","[-0.007978754118084908, 0.005621972493827343, ..."
3,3,000001KP1,RUSSELL 1000 INDEX,0.0,NaN,RLG,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR,NaN,NaN,NaN,"RUSSELL 1000 INDEX,0,NULL,RLG,EMI,UNKNOWN,UNKN...","[-0.009771451354026794, 0.006706769112497568, ..."
4,4,000001KR7,RUSSEL 3000 INDEX,0.0,NaN,RLG,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR,NaN,NaN,NaN,"RUSSEL 3000 INDEX,0,NULL,RLG,EMI,UNKNOWN,UNKNO...","[-0.010332748293876648, 0.00992654636502266, -..."
